In [7]:
import pandas as pd 
import os
import datetime as dt
import re

In [8]:
def toSigned(string):
    m = re.match(r"([1-9.]+?)([A-Z])", string)
    num_string = m.group(1)
    cardinal = m.group(2)
    num = float(num_string)

    if(cardinal == "S" or cardinal == "W"):
        num *= -1

    return num

In [9]:
geoDF = pd.read_csv('./geodata.csv', index_col = 0)
geoDF.head()

,name,Location,Coordinates,Landscape,Climate,Reference,Site PI,Data Available
Site ID,,,,,,,,
1,Bermuda Island,"Bermuda Island, England","32°15’N, 64°52’W",Coast,Dwa,Steen-Larsen et al. (2015),Hans Christian Steen-Larsen,In prep.
2,Beijing,"Beijing, China","40°0’N, 116°3’E",Urban,Dwa,Wen et al. (2010),Xuhui Lee and Xuefa Wen,Yes
3,caborden,"Borden, Canada","44°9′N, 79°6′W",forest,Dfb,Santos et al. (2012),Xuhui Lee,Yes
4,Camargue,"Vaccarès lagoon, France","43°35’N, 4°28’E",Wetland,Csb,Delattre et al. (2015),Christine Vallet-Coulomb,Yes
5,Dome C,"Concordia, South Pole","75°06′S, 123°23′E",Ice Sheet,-,Casado et al (2016),Mathieu Casado,Yes


In [10]:
geoDF.name= geoDF.name.astype(str)
geoDF.dtypes

name              object
Location          object
Coordinates       object
Landscape         object
Climate           object
Reference         object
Site PI           object
Data Available    object
dtype: object

In [4]:
isodfs = []
isopath = "./isotope-data/"
for filename in os.listdir(isopath):
    extension = filename[-4:]
    if(extension == '.csv'):
        country, name = filename[:-4].split('-')
        df = pd.read_csv(isopath + filename, encoding = "iso-8859-1")
        if 'Delta_18O' in df.columns and 'Delta_D' in df.columns:
            df = df[df['Delta_18O'] != -9999]
            df = df[df['Delta_D'] != -9999]
            df['Country'] = country
            df['Name'] = name
            if(len(df.index) > 0):
                df['TIMESTAMP'] = df['TIMESTAMP'].astype(int).astype(str)
                isodfs.append(df)
        


KeyError: 'TIMESTAMP'

In [11]:
isodfs[0].head()

,TIMESTAMP,Delta_18O,Delta_D,Latitude,Longitude,Landscape,Country,Name
0,201306181500,-13.640750,-126.381000,28.3N,16.5W,Mountain,SP,Teide
1,201306181600,-15.774167,-145.946000,28.3N,16.5W,Mountain,SP,Teide
2,201306181700,-20.673000,-182.658333,28.3N,16.5W,Mountain,SP,Teide
3,201306181800,-25.917500,-223.841500,28.3N,16.5W,Mountain,SP,Teide
4,201306181900,-26.898667,-231.194333,28.3N,16.5W,Mountain,SP,Teide


In [12]:
megadf = pd.concat(isodfs, ignore_index=True).filter(['TIMESTAMP', 'Delta_18O','Delta_D', 'Latitude', 'Longitude', 'Landscape', 'Country', 'Name'])
megadf.dtypes

TIMESTAMP     object
Delta_18O    float64
Delta_D      float64
Latitude      object
Longitude     object
Landscape     object
Country       object
Name          object
dtype: object

In [13]:
megadf["sLatitude"] = megadf["Latitude"].apply(toSigned)
megadf["sLongitude"] = megadf["Longitude"].apply(toSigned)

AttributeError: 'NoneType' object has no attribute 'group'

In [16]:
megadf.to_csv('./mega.csv')